In [6]:
import os
os.makedirs('a')

In [18]:
def check_dir(filename):
    original = filename
    file_order = 1
    
    while True:
        if os.path.isdir(filename):
            new_name = original + str(file_order)
            filename = new_name
            file_order += 1
        else:
            os.makedirs(filename)
            break
check_dir('a2')

In [4]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

import json
from urllib.request import urlopen
import time
import os

caps = DesiredCapabilities.CHROME
caps['loggingPrefs'] = {'performance': 'ALL'}
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(desired_capabilities=caps,options=options)

#driver = webdriver.Chrome('Chromedriver.exe')

base_url = 'https://search.naver.com/search.naver?where=image&sm=tab_jum&query='
food_list_kr = ['아메리카노', '카푸치노', '딸기라떼', '녹차라떼']
food_list_en = ['americano', 'cappucino', 'latte_strawberry', 'latte_greentea']

print('아래의 음식 사진을 수집합니다.')
print(food_list_kr)

image_total = 0
os.makedirs('new_data', exist_ok=True)
for food_en, food_kr in zip(food_list_en, food_list_kr):
    print('-----------------------', food_kr + '  시작------------------')
    driver.get(base_url + food_kr)
    delay = 10
    
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.ID, 'main_pack')))
        print("페이지 로딩 완료")
    except TimeoutException:
        print("페이지 로딩 실패")
        break
        
    list_len_for_scroll = 0
    while True:
        driver.find_element_by_tag_name('html').send_keys(Keys.END)
        time.sleep(0.7)
        driver.find_element_by_class_name('photo_tile')
        url_list_for_scroll = driver.find_elements_by_class_name('_image')
        
        
        if len(url_list_for_scroll) == list_len_for_scroll:
            driver.find_element_by_tag_name('html').send_keys(Keys.HOME)
            y = 1000
            for timer in range(0,50):
                driver.execute_script("window.scrollTo(0, "+str(y)+")")
                y += 1000  
                time.sleep(0.1)
            break

        list_len_for_scroll = len(url_list_for_scroll)
        
    driver_log = driver.get_log('performance')
    image_url_list = []
    for i, log in enumerate(driver_log):
        if 'request' in json.loads(log['message'])['message']['params']:
            url = json.loads(log['message'])['message']['params']['request']['url']
            if 'https://search.pstatic.net/common/' in url:
                image_url_list.append(url)   
    
    count = 1
    os.makedirs('new_data\\' + food_en, exist_ok=True)
    for image_url in set(image_url_list):
        with urlopen(image_url) as f:
            with open('{}\{}\{}'.format('new_data', food_en, food_en) + str(count) + '.jpg', 'wb') as file_name:
                img = f.read()
                file_name.write(img)
        count += 1
        
    print(food_kr + ' 사진 개수 : {}개'.format(len(image_url_list)))
    print('-----------------------', food_kr + '  종료------------------')
    image_total += len(image_url_list)

print('크롤링 끝')
print('수집된 이미지의 총 개수는 {}개입니다.'.format(image_total))

아래의 음식 사진을 수집합니다.
['아메리카노', '카푸치노', '딸기라떼', '녹차라떼']
----------------------- 아메리카노  시작------------------
페이지 로딩 완료
아메리카노 사진 개수 : 487개
----------------------- 아메리카노  종료------------------
----------------------- 카푸치노  시작------------------
페이지 로딩 완료
카푸치노 사진 개수 : 488개
----------------------- 카푸치노  종료------------------
----------------------- 딸기라떼  시작------------------
페이지 로딩 완료


KeyboardInterrupt: 